Impotação das bibliotecas necessárias

In [40]:
import os
import cv2
import numpy as np
import pandas as pd
from scipy.stats import skew, kurtosis, entropy

Função para calcular os parâmetros do histograma

In [41]:
def calcular_parametros_histograma(image):
    # Verifica se a imagem foi carregada corretamente
    if image is None:
        print("Erro: A imagem não foi carregada corretamente.")
        return None
    
    # Converte para o espaço de cor HSV
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    
    # Define intervalo para cores escuras (ajuste os valores conforme necessário)
    lower_dark = np.array([0, 150, 40])
    upper_dark = np.array([179, 255, 150])
    
    # Cria uma máscara para as áreas escuras
    mask = cv2.inRange(hsv, lower_dark, upper_dark)
    
    # Aplica uma operação de morfologia para limpar a máscara
    kernel = np.ones((3, 3), np.uint8)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel, iterations=2)
    
    # Aplica a máscara na imagem para isolar as lesões
    isolated_lesion = cv2.cvtColor(image, cv2.COLOR_BGR2BGRA)
    isolated_lesion[:, :, 3] = mask  # O canal alfa é definido pela máscara
    
    # Agora, utilizamos a máscara para calcular o histograma apenas nas áreas isoladas
    isolated_area = cv2.bitwise_and(image, image, mask=mask)
    
    # Verifica se há pixels isolados na área para evitar erro de índice
    isolated_area_values = isolated_area[mask == 255]
    if isolated_area_values.size == 0:
        return None  # Retorna None se não houver áreas isoladas
    
    # Calcular o histograma da área isolada (apenas nas regiões escuras)
    hist, bins = np.histogram(isolated_area_values.ravel(), bins=256, range=(0, 256))
    
    # Calcular parâmetros do histograma
    mean_intensity = np.mean(isolated_area_values)  # Média nas áreas isoladas
    percentile_50 = np.percentile(isolated_area_values, 50)  # Percentil 50
    iqr = np.percentile(isolated_area_values, 75) - np.percentile(isolated_area_values, 25)  # Intervalo Interquartil
    intensity_skewness = skew(isolated_area_values.ravel())  # Skewness nas áreas isoladas
    intensity_kurtosis = kurtosis(isolated_area_values.ravel())  # Kurtosis nas áreas isoladas
    intensity_entropy = entropy(hist)  # Entropia do histograma
    
    # Retornar os parâmetros como um dicionário
    return {
        "Média": mean_intensity,
        "Percentil 50": percentile_50,
        "IQR": iqr,
        "Skewness": intensity_skewness,
        "Kurtosis": intensity_kurtosis,
        "Entropia": intensity_entropy
    }

Caminho para o diretório de imagens

In [42]:
dataset_path = './Skin Cancer'

Contador de imagens sem áreas isoladas

In [43]:
count_no_isolated_area = 0

Lista para armazenar os resultados

In [44]:
resultados = []

Iterar imagens

In [45]:
for root, dirs, files in os.walk(dataset_path):
    for file in files:
        if file.endswith(('.jpg', '.jpeg', '.png')):  # Considerar apenas arquivos de imagem
            # Caminho completo da imagem
            image_path = os.path.join(root, file)
            
            # Carregar a imagem em cor (não mais em escala de cinza)
            image = cv2.imread(image_path)
            
            if image is not None:
                # Calcular os parâmetros do histograma
                parametros = calcular_parametros_histograma(image)
                
                if parametros:
                    parametros["Imagem"] = file  # Adicionar o nome do arquivo à linha de resultados
                
                    # Adicionar os resultados à lista
                    resultados.append(parametros)
                else:
                    # Incrementa o contador quando não há áreas isoladas
                    count_no_isolated_area += 1

Criar um DataFrame com os resultados

In [46]:
df_resultados = pd.DataFrame(resultados)

Salvar em .csv

In [47]:
df_resultados.to_csv('resultados_histograma_skin_cancer.csv', index=False)

Exibe o total de imagens sem áreas isoladas

In [48]:
print(f"Total de imagens sem áreas isoladas: {count_no_isolated_area}")

Total de imagens sem áreas isoladas: 986


Carrega os arquivos .csv para mesclagem

In [49]:
df_histograma = pd.read_csv('resultados_histograma_skin_cancer.csv')
df_metadata = pd.read_csv('HAM10000_metadata.csv')

Renomear a coluna "Imagem" para "image_id" para facilitar a comparação

In [50]:
df_histograma.rename(columns={"Imagem": "image_id"}, inplace=True)

Remover a extensão '.jpg' da coluna 'image_id' no df_histograma

In [51]:
df_histograma['image_id'] = df_histograma['image_id'].str.replace('.jpg', '', regex=False)

Realizar a mesclagem entre os DataFrames com base na coluna 'image_id'

In [52]:
df_completo = pd.merge(df_metadata, df_histograma, on='image_id', how='inner')
df_completo

,lesion_id,image_id,dx,dx_type,age,sex,localization,Média,Percentil 50,IQR,Skewness,Kurtosis,Entropia
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp,86.625786,66.0,73.75,0.705317,-1.278666,4.020837
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp,78.500000,61.0,70.25,0.741633,-0.892863,3.650720
2,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp,73.333333,58.0,41.00,0.781195,-0.735490,2.523211
3,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear,83.363868,68.0,75.00,0.559697,-1.230686,4.436134
4,HAM_0001466,ISIC_0027850,bkl,histo,75.0,male,ear,37.090213,35.0,25.00,2.397979,9.735966,4.029376
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9024,HAM_0005705,ISIC_0031430,akiec,histo,75.0,female,lower extremity,63.803704,61.0,49.00,0.249045,-1.182563,4.386614
9025,HAM_0002867,ISIC_0033084,akiec,histo,40.0,male,abdomen,70.049708,60.0,55.00,0.561039,-0.990349,4.417310
9026,HAM_0002867,ISIC_0033550,akiec,histo,40.0,male,abdomen,50.827765,46.0,34.50,1.179570,1.399920,4.504961
9027,HAM_0002867,ISIC_0033536,akiec,histo,40.0,male,abdomen,76.564103,65.0,61.50,0.615415,-1.047779,3.414740


Salvar o DataFrame resultante

In [53]:
df_completo.to_csv('HAM10000_metadata_atualizado.csv', index=False)

Carregar o arquivo CSV para mudar a ordem das colunas

In [54]:
df = pd.read_csv('HAM10000_metadata_atualizado.csv')

Mover a coluna 'dx' para a última posição

In [55]:
cols = [col for col in df.columns if col != 'dx'] + ['dx']
df = df[cols]
df

,lesion_id,image_id,dx_type,age,sex,localization,Média,Percentil 50,IQR,Skewness,Kurtosis,Entropia,dx
0,HAM_0000118,ISIC_0027419,histo,80.0,male,scalp,86.625786,66.0,73.75,0.705317,-1.278666,4.020837,bkl
1,HAM_0000118,ISIC_0025030,histo,80.0,male,scalp,78.500000,61.0,70.25,0.741633,-0.892863,3.650720,bkl
2,HAM_0002730,ISIC_0025661,histo,80.0,male,scalp,73.333333,58.0,41.00,0.781195,-0.735490,2.523211,bkl
3,HAM_0001466,ISIC_0031633,histo,75.0,male,ear,83.363868,68.0,75.00,0.559697,-1.230686,4.436134,bkl
4,HAM_0001466,ISIC_0027850,histo,75.0,male,ear,37.090213,35.0,25.00,2.397979,9.735966,4.029376,bkl
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9024,HAM_0005705,ISIC_0031430,histo,75.0,female,lower extremity,63.803704,61.0,49.00,0.249045,-1.182563,4.386614,akiec
9025,HAM_0002867,ISIC_0033084,histo,40.0,male,abdomen,70.049708,60.0,55.00,0.561039,-0.990349,4.417310,akiec
9026,HAM_0002867,ISIC_0033550,histo,40.0,male,abdomen,50.827765,46.0,34.50,1.179570,1.399920,4.504961,akiec
9027,HAM_0002867,ISIC_0033536,histo,40.0,male,abdomen,76.564103,65.0,61.50,0.615415,-1.047779,3.414740,akiec


Salvar o DataFrame resultante em um novo arquivo CSV

In [56]:
df.to_csv('HAM10000_metadata_reorganizado.csv', index=False)